In [ ]:
# Imports

In [1]:
%%time
#Imports requisite packages
import os
import time
import numpy
import pickle
import cProfile
import itertools
import matplotlib
import sklearn.tree
import sklearn.metrics
import sklearn.ensemble
import sklearn.preprocessing
import sklearn.learning_curve
import sklearn.model_selection
import sklearn.cross_validation
import sklearn.feature_selection
import sklearn.kernel_approximation
from matplotlib import pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.cross_validation import *
from sklearn.metrics import *
from sklearn.feature_selection import *
from sklearn.preprocessing import *
from sklearn.model_selection import *


#%jsroot on9
%matplotlib inline
matplotlib.use('Agg')

/usr/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/anaconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


CPU times: user 556 ms, sys: 60 ms, total: 616 ms
Wall time: 895 ms


/usr/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


# Function Definitions

In [2]:
%%time
#Takes the converted tree and turns it into an
#n-by-30 array usable by sklearn.
def outputs(array):
    #Only uses events with non-zero luminosity
    goodEvents = array[array['lumi'] != 0]
    ind = numpy.lexsort((goodEvents['lumiId'],goodEvents['runId']))
    events = goodEvents[ind]
    dataset = numpy.empty([len(goodEvents),30])
    target = numpy.empty([len(goodEvents)])
    badOnes = numpy.array([])

    #Fills dataset array with proper features
    for j, event in enumerate(events):
        try:
            dataset[j,0:7] = event['qPFJetPt']
            dataset[j,7:14] = event['qPFJetEta']
            dataset[j,14:21] = event['qPFJetPhi']
            dataset[j,21:28] = event['qNVtx']
            dataset[j,28] = event['crossSection']
            dataset[j,29] = event['lumi']
            target[j] = event['isSig']
        except ValueError:
            badOnes = numpy.append(badOnes,j)
            
    #Takes out corrupt events
    mask = numpy.zeros(len(dataset), dtype=bool)
    mask[badOnes.astype(int)] = True
    mask = ~mask
    dataset = dataset[mask]
    target = target[mask]
       
    return dataset, target

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.96 µs


In [3]:
#Function that plots confusion matrix, taken from sklearn website
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    #This function prints and plots the confusion matrix.
    #Normalization can be applied by setting `normalize=True`.     
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, numpy.newaxis]
        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title)
        plt.colorbar()
        tick_marks = numpy.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45)
        plt.yticks(tick_marks, classes)
        print("Normalized confusion matrix")
    else:
        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title)
        plt.colorbar()
        tick_marks = numpy.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45)
        plt.yticks(tick_marks, classes)
        print('Confusion matrix, without normalization')

    print(cm)
    
    thresh = cm.max()*.7
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Data Prep

In [4]:
#Loads pickled dataset
inFile = open('realData.pkl', 'rb')
dataset = pickle.load(inFile, encoding="latin1")
target = pickle.load(inFile, encoding="latin1")
inFile.close()

In [5]:
lumi = dataset[:,-1]
dataset = dataset[:,:-1]
sigInd = numpy.where(target==1)
backInd = numpy.where(target==0)
sigVals = dataset[sigInd]
backVals = dataset[backInd]
sigTarget = target[sigInd]
backTarget = target[backInd]

sigTrain, sigTest, sigTrainTarget, sigTestTarget = train_test_split(sigVals, sigTarget, test_size=.5)

In [6]:
# #Imports training and testing sets used across models
inFile = open('splits.pkl', 'rb')
xTrain = pickle.load(inFile, encoding="latin1")
xTest = pickle.load(inFile, encoding="latin1")
yTrain = pickle.load(inFile, encoding="latin1")
yTest = pickle.load(inFile, encoding="latin1")
lumiTrain = pickle.load(inFile, encoding="latin1")
lumiTest = pickle.load(inFile, encoding="latin1")
inFile.close()

In [7]:
#Removes features with variance less than 0.1
sel = VarianceThreshold(threshold = 0.01)
print(sigTrain.shape)
sel.fit(sigTrain)
indices = sel.get_support()
sigTrain = sigTrain[:,indices]
print(sigTrain.shape)
sigTest = sigTest[:,indices]
print(indices)
backVals = backVals[:,indices]

(110220, 29)
(110220, 14)
[ True  True False  True  True  True  True False False False False False
 False False False False False False False False False  True  True  True
  True  True  True  True  True]


In [8]:
sigTes

# Tuning hyper-parameters for precision



KeyboardInterrupt: 

In [ ]:
#Scales the data to zero mean and unit variance
scaler = StandardScaler()
scaler.fit(sigTrain)
sigTrain = scaler.transform(sigTrain)
sigTest = scaler.transform(sigTest)
backVals = scaler.transform(backVals)
backTarget[backTarget == 0] = -1
# sigTrainTarget[sigTrainTarget == 0] = -1
# sigTestTarget[sigTestTarget == 0] = -1
# print(len(sigTestTarget[sigTestTarget==-1]))
# print(len(sigTrainTarget[sigTrainTarget==1]))

In [ ]:
%%time
isfClf = IsolationForest(n_jobs=-1, contamination = 0.1, max_samples = 100, n_estimators = 500)
isfClf.fit(sigTrain)

In [ ]:
%%time

#Provides classification reports 
isfScore = isfClf.fit(sigTrain).decision_function(sigTest)
    
isfPredict = isfClf.predict(sigTest)
print("Classification report for ISF, Tuned, Weights %s:\n%s\n"
      % (isfClf, sklearn.metrics.classification_report(sigTestTarget, isfPredict)))

In [ ]:
# %%time
#Plots classification results for signal and background
isfArrs = []
isfHists = []

#Separates decision function results into signal and background
#along with training and testing
isfArrs.append(isfClf.decision_function(sigTrain).ravel())
isfArrs.append(isfClf.decision_function(sigTest).ravel())
isfArrs.append(isfClf.decision_function(backVals).ravel())
print(len(isfArrs[0]))
print(len(isfArrs[1]))
print(len(isfArrs[2]))
    

#Turns those arrays into histograms
isfHists.append(list(numpy.histogram(isfArrs[0], normed = True, bins = 40)))
isfHists.append(list(numpy.histogram(isfArrs[1], normed = True, bins = 40)))
isfHists.append(list(numpy.histogram(isfArrs[2], normed = True, bins = 40)))

#Defines bin edges, centers, and widths
isfMax = max([hist[0].max() for hist in isfHists])*1.2
isfMin = max([hist[0].min() for hist in isfHists])
isfEdges = isfHists[0][1]
isfCenters = (isfEdges[:-1] + isfEdges[1:])/2.
isfWidths = (isfEdges[1:] - isfEdges[:-1])

In [ ]:
#Normalizes histogram based on maximum value
isfNormVal1 = max(max(isfHists[0][0]), max(isfHists[1][0]), max(isfHists[2][0]))
isfHists[0][0] = [x/isfNormVal1 for x in isfHists[0][0]]
isfHists[1][0] = [x/isfNormVal1 for x in isfHists[1][0]]
isfHists[2][0] = [x/isfNormVal1 for x in isfHists[2][0]]

In [ ]:
%%time
#Plots histograms
ax1 = plt.subplot(111)
ax1.bar(isfCenters-isfWidths/2.,isfHists[0][0],facecolor='red',linewidth=0,width=isfWidths,label='SignalTrain',alpha=0.5)
ax1.bar(isfCenters-isfWidths/2.,isfHists[1][0],facecolor='yellow',linewidth=0,width=isfWidths,label='SignalTest',alpha=0.5)
ax1.bar(isfCenters-isfWidths/2.,isfHists[2][0],facecolor='blue',linewidth=0,width=isfWidths,label='Background',alpha=0.5)
#Change depending on which classifier and options are chosen
plt.title("Classification, isf, Tuned, Weights, Rand, 15 feats, Training Set")
plt.xlabel("classifier score")
plt.ylabel("Counts/Bin")
legend = ax1.legend(loc='upper center', shadow=True,ncol=3)
for alabel in legend.get_texts():
            alabel.set_fontsize('small')
plt.legend(loc='upper left')
plt.show()

# ax2 = plt.subplot(111)
# ax2.bar(isfCenters-isfWidths/2.,isfHists[2][0],facecolor='red',linewidth=0,width=isfWidths,label='Signal',alpha=0.5)
# ax2.bar(isfCenters-isfWidths/2.,isfHists[3][0],facecolor='blue',linewidth=0,width=isfWidths,label='Background',alpha=0.5)
# plt.title("Classification, isf, Tuned, Weights, Rand, 15 feats, Testing Set")
# plt.xlabel("classifier score")
# plt.ylabel("Counts/Bin")
# legend = ax1.legend(loc='upper center', shadow=True,ncol=2)
# for alabel in legend.get_texts():
#             alabel.set_fontsize('small')
# plt.legend(loc='upper left')
# plt.show()

In [ ]:
# # %%time
# #Plots classification results for signal and background
# isfArrs = []
# isfHists = []

# #Separates decision function results into signal and background
# #along with training and testing
# isfArrs.append(isfClf.decision_function(xTrain[yTrain>0.5]).ravel())
# isfArrs.append(isfClf.decision_function(xTrain[yTrain<0.5]).ravel())
# isfArrs.append(isfClf.decision_function(xTest[yTest>0.5]).ravel())
# isfArrs.append(isfClf.decision_function(xTest[yTest<0.5]).ravel())

# #Turns those arrays into histograms
# isfHists.append(list(numpy.histogram(isfArrs[0], normed = True, bins = 40)))
# isfHists.append(list(numpy.histogram(isfArrs[1], normed = True, bins = 40)))
# isfHists.append(list(numpy.histogram(isfArrs[2], normed = True, bins = 40)))
# isfHists.append(list(numpy.histogram(isfArrs[3], normed = True, bins = 40)))

# #Defines bin edges, centers, and widths
# isfMax = max([hist[0].max() for hist in isfHists])*1.2
# isfMin = max([hist[0].min() for hist in isfHists])
# isfEdges = isfHists[0][1]
# isfCenters = (isfEdges[:-1] + isfEdges[1:])/2.
# isfWidths = (isfEdges[1:] - isfEdges[:-1])

In [ ]:
# #Normalizes histogram based on maximum value
# isfNormVal1 = max(max(isfHists[0][0]), max(isfHists[1][0]))
# isfNormVal2 = max(max(isfHists[2][0]), max(isfHists[3][0]))
# isfHists[0][0] = [x/isfNormVal1 for x in isfHists[0][0]]
# isfHists[1][0] = [x/isfNormVal1 for x in isfHists[1][0]]
# isfHists[2][0] = [x/isfNormVal2 for x in isfHists[2][0]]
# isfHists[3][0] = [x/isfNormVal2 for x in isfHists[3][0]]

In [ ]:
# %%time
# #Plots histograms
# ax1 = plt.subplot(111)
# ax1.bar(isfCenters-isfWidths/2.,isfHists[0][0],facecolor='red',linewidth=0,width=isfWidths,label='Signal',alpha=0.5)
# ax1.bar(isfCenters-isfWidths/2.,isfHists[1][0],facecolor='blue',linewidth=0,width=isfWidths,label='Background',alpha=0.5)
# #Change depending on which classifier and options are chosen
# plt.title("Classification, isf, Tuned, Weights, Rand, 15 feats, Training Set")
# plt.xlabel("classifier score")
# plt.ylabel("Counts/Bin")
# legend = ax1.legend(loc='upper center', shadow=True,ncol=2)
# for alabel in legend.get_texts():
#             alabel.set_fontsize('small')
# plt.legend(loc='upper left')
# plt.show()

# ax2 = plt.subplot(111)
# ax2.bar(isfCenters-isfWidths/2.,isfHists[2][0],facecolor='red',linewidth=0,width=isfWidths,label='Signal',alpha=0.5)
# ax2.bar(isfCenters-isfWidths/2.,isfHists[3][0],facecolor='blue',linewidth=0,width=isfWidths,label='Background',alpha=0.5)
# plt.title("Classification, isf, Tuned, Weights, Rand, 15 feats, Testing Set")
# plt.xlabel("classifier score")
# plt.ylabel("Counts/Bin")
# legend = ax1.legend(loc='upper center', shadow=True,ncol=2)
# for alabel in legend.get_texts():
#             alabel.set_fontsize('small')
# plt.legend(loc='upper left')
# plt.show()

In [ ]:
%%time
#Plots roc curve, code taken from sklearn website
fpr, tpr, _ = roc_curve(yTest, isfScore)
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2;
plt.plot(fpr, tpr, color='darkorange',
        lw = lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw = lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC isf, Tuned, Weights, Rand, 15 feat')
plt.legend(loc="lower right")
plt.show()

In [ ]:
%%time
#Plots confusion matrix, code taken from sklearn
classNames = ['Background','Signal']
confMat = confusion_matrix(yTest, isfPredict)
#numpy.set_printoptions(precision=)

#Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(confMat, classes=classNames,
                      title='Confusion matrix, isf, Tuned, Un-normalization, Weights')

#Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(confMat, classes=classNames, normalize=True,
                      title='Confusion Matrix, isf, Tuned, Normalized, Weights')

plt.show()

In [ ]:
%%time
#Calculates Matthews Correlation Coefficient
#Ranges from -1 to 1, with 1 being a perfect predictor
matthews_corrcoef(yTest,isfPredict)